# Chapter 21

## Structured Streaming: Basics

In [1]:
static = spark.read.json("gs://reddys-data-for-experimenting/activity-data/")
dataSchema = static.schema
static.count()

6240991

In [2]:
from pyspark.sql.types import StructField, TimestampType
dataSchema.fields[1] = StructField("Creation_Time", TimestampType(), True)
static.printSchema()
print(dataSchema)

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)

StructType(List(StructField(Arrival_Time,LongType,true),StructField(Creation_Time,TimestampType,true),StructField(Device,StringType,true),StructField(Index,LongType,true),StructField(Model,StringType,true),StructField(User,StringType,true),StructField(gt,StringType,true),StructField(x,DoubleType,true),StructField(y,DoubleType,true),StructField(z,DoubleType,true)))


### Reading a stream

In [3]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json("gs://reddys-data-for-experimenting/activity-data/")

In [4]:
transformedStream = streaming.selectExpr("User", "gt", "Model")

In [5]:
transformedStreamWriter = transformedStream.writeStream \
    .option("checkpointLocation", "gs://reddys-data-for-experimenting/output/chkpnt") \
    .queryName("transformedStream")\
    .outputMode("append")\
    .start(path="gs://reddys-data-for-experimenting/output/streaming", format="parquet") 
    # .awaitTermination()

### Monitoring your streams Application

In [6]:
transformedStreamWriter.status

{u'isDataAvailable': True,
 u'isTriggerActive': False,
 u'message': u'Processing new data'}

In [7]:
transformedStreamWriter.recentProgress

[{u'batchId': 0,
  u'durationMs': {u'addBatch': 4722,
   u'getBatch': 239,
   u'getOffset': 1081,
   u'queryPlanning': 60,
   u'triggerExecution': 6874,
   u'walCommit': 711},
  u'id': u'9abb1cc1-5e13-4472-ae6b-eb0063f65ce2',
  u'name': u'transformedStream',
  u'numInputRows': 78012,
  u'processedRowsPerSecond': 11343.899956376326,
  u'runId': u'b208329b-1676-4851-9c0a-ab1d5b458bd5',
  u'sink': {u'description': u'FileSink[gs://reddys-data-for-experimenting/output/streaming]'},
  u'sources': [{u'description': u'FileStreamSource[gs://reddys-data-for-experimenting/activity-data]',
    u'endOffset': {u'logOffset': 0},
    u'numInputRows': 78012,
    u'processedRowsPerSecond': 11343.899956376326,
    u'startOffset': None}],
  u'stateOperators': [],
  u'timestamp': u'2019-10-06T15:46:24.930Z'},
 {u'batchId': 1,
  u'durationMs': {u'addBatch': 7114,
   u'getBatch': 134,
   u'getOffset': 827,
   u'queryPlanning': 27,
   u'triggerExecution': 9443,
   u'walCommit': 1336},
  u'id': u'9abb1cc1-5e13

### Validating if the stream is working as expected

In [8]:
spark.streams.active

In [13]:
staticTransformedData = spark.read.format("parquet").load("gs://reddys-data-for-experimenting/output/streaming")

In [14]:
staticTransformedData.printSchema()

root
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- Model: string (nullable = true)



In [15]:
staticTransformedData.count()

6240991

In [16]:
staticTransformedData.describe().show()

+-------+-------+-------+-------+
|summary|   User|     gt|  Model|
+-------+-------+-------+-------+
|  count|6240991|6240991|6240991|
|   mean|   null|   null|   null|
| stddev|   null|   null|   null|
|    min|      a|   bike| nexus4|
|    max|      i|   walk| nexus4|
+-------+-------+-------+-------+



### Summay:

Input data statics meets output data stats with data fromat converted to `Parquet` and with some stateless transformation.